## Import Dependencies

In [9]:
# dependencies
from flask import Flask, render_template, jsonify
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

## Scrape the cash rate data from the RBA website and store in a Pandas Dataframe

In [10]:
# Make a request to the RBA website
url = 'https://www.rba.gov.au/statistics/cash-rate/'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the interest rate data
table = soup.find('table', {'id': 'datatable','class': 'table-linear table-numeric'})

# Initialize an empty list to store the data
data = []

# Extract the data from the table
for row in table.find_all('tr'):
    cells = row.find_all(['th','td'])
    if len(cells) == 4:
        date = cells[0].text.strip()
        change = cells[1].text.strip()
        rate = cells[2].text.strip()
        data.append([date, change, rate])
    else:
        continue
        
# Create a pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])
df.columns = ["date", "change_pct", "cash_rate_pct"]

# Print the DataFrame
print(df)

            date      change_pct   cash_rate_pct
0     7 Dec 2022           +0.25            3.10
1     2 Nov 2022           +0.25            2.85
2     5 Oct 2022           +0.25            2.60
3     7 Sep 2022           +0.50            2.35
4     3 Aug 2022           +0.50            1.85
..           ...             ...             ...
357   2 May 1990            0.00  15.00 to 15.50
358   4 Apr 1990  -1.00 to -1.50  15.00 to 15.50
359   7 Mar 1990            0.00  16.50 to 17.00
360  15 Feb 1990           -0.50  16.50 to 17.00
361  23 Jan 1990  -0.50 to -1.00  17.00 to 17.50

[362 rows x 3 columns]


## 2.	Use the SQLAlchemy library to connect to a SQL database and create a table to store the scraped data.

In [13]:
app = Flask(__name__)

# connect to database using SQLAlchemy
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'rates_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

# Confirm tables
insp.get_table_names()

# Load DataFrame into Postgres Database
df.to_sql(name='interest_rates', con=engine, if_exists='append', index=False)

# Test Database with SQL Query
pd.read_sql_query('SELECT * from interest_rates LIMIT 100', con=engine).head()

,id,date,change_pct,cash_rate_pct
0,1,2022-12-07,+0.25,3.10
1,2,2022-11-02,+0.25,2.85
2,3,2022-10-05,+0.25,2.60
3,4,2022-09-07,+0.50,2.35
4,5,2022-08-03,+0.50,1.85


In [14]:
# define a route to retrieve data from the database and serve it as a JSON response
@app.route('/api/data')
def data():
    # Query the database
    query = 'SELECT date, change_pct, cash_rate_pct FROM interest_rates'
    df = pd.read_sql_query(query, con=engine)
    # Convert the data to a JSON object
    data = df.to_json(orient='records')
    return jsonify(data)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\rober\anaconda\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
